In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression


pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../../../archive/blazed_diamond_ranked_10min.csv")

df.head()

,blueWins,blueWardsDestroyed,blueFirstBlood,blueKills,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueAvgLevel,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsDestroyed,redKills,redAssists,redDragons,redHeralds,redTowersDestroyed,redAvgLevel,redTotalJungleMinionsKilled,redCSPerMin,redGoldPerMin,blueVisionScore,redVisionScore
0,0,2,1,9,11,0,0,0,6.6,36,643,-8,19.5,1721.0,6,6,8,0,0,0,6.8,55,19.7,1656.7,28,15
1,0,1,0,5,5,0,0,0,6.6,43,-2908,-1173,17.4,1471.2,1,5,2,1,1,1,6.8,52,24.0,1762.0,12,12
2,0,0,0,7,4,1,0,0,6.4,46,-1172,-1033,18.6,1611.3,3,11,14,0,0,0,6.8,28,20.3,1728.5,15,15
3,0,1,0,4,5,0,1,0,7.0,55,-1321,-7,20.1,1515.7,2,5,10,0,0,0,7.0,47,23.5,1647.8,43,15
4,0,4,0,6,6,0,0,0,7.0,57,-1004,230,21.0,1640.0,2,6,7,1,0,0,7.0,67,22.5,1740.4,75,17


In [3]:
df.describe()

,blueWins,blueWardsDestroyed,blueFirstBlood,blueKills,blueAssists,blueDragons,blueHeralds,blueTowersDestroyed,blueAvgLevel,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin,redWardsDestroyed,redKills,redAssists,redDragons,redHeralds,redTowersDestroyed,redAvgLevel,redTotalJungleMinionsKilled,redCSPerMin,redGoldPerMin,blueVisionScore,redVisionScore
count,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000,9879.000000
mean,0.499038,2.824881,0.504808,6.183925,6.645106,0.361980,0.187974,0.051422,6.916004,50.509667,14.414111,-33.620306,21.669956,1650.345551,2.723150,6.137666,6.662112,0.413098,0.160036,0.043021,6.925316,51.313088,21.734923,1648.904140,22.288288,22.367952
std,0.500024,2.174998,0.500002,3.011028,4.064520,0.480597,0.390712,0.244369,0.305146,9.898282,2453.349179,1920.370438,2.185844,153.544664,2.138356,2.933818,4.060612,0.492415,0.366658,0.216900,0.305311,10.027885,2.191167,149.088841,18.019177,18.457427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.600000,0.000000,-10830.000000,-9333.000000,9.000000,1073.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.800000,4.000000,10.700000,1121.200000,5.000000,6.000000
25%,0.000000,1.000000,0.000000,4.000000,4.000000,0.000000,0.000000,0.000000,6.800000,44.000000,-1585.500000,-1290.500000,20.200000,1541.550000,1.000000,4.000000,4.000000,0.000000,0.000000,0.000000,6.800000,44.000000,20.300000,1542.750000,14.000000,14.000000
50%,0.000000,3.000000,1.000000,6.000000,6.000000,0.000000,0.000000,0.000000,7.000000,50.000000,14.000000,-28.000000,21.800000,1639.800000,2.000000,6.000000,6.000000,0.000000,0.000000,0.000000,7.000000,51.000000,21.800000,1637.800000,16.000000,16.000000
75%,1.000000,4.000000,1.000000,8.000000,9.000000,1.000000,0.000000,0.000000,7.200000,56.000000,1596.000000,1212.000000,23.200000,1745.900000,4.000000,8.000000,9.000000,1.000000,0.000000,0.000000,7.200000,57.000000,23.300000,1741.850000,20.000000,20.000000
max,1.000000,27.000000,1.000000,22.000000,29.000000,1.000000,1.000000,4.000000,8.000000,92.000000,11467.000000,8348.000000,28.300000,2370.100000,24.000000,22.000000,28.000000,1.000000,1.000000,2.000000,8.200000,92.000000,28.900000,2273.200000,250.000000,276.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9879 entries, 0 to 9878
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   blueWins                      9879 non-null   int64  
 1   blueWardsDestroyed            9879 non-null   int64  
 2   blueFirstBlood                9879 non-null   int64  
 3   blueKills                     9879 non-null   int64  
 4   blueAssists                   9879 non-null   int64  
 5   blueDragons                   9879 non-null   int64  
 6   blueHeralds                   9879 non-null   int64  
 7   blueTowersDestroyed           9879 non-null   int64  
 8   blueAvgLevel                  9879 non-null   float64
 9   blueTotalJungleMinionsKilled  9879 non-null   int64  
 10  blueGoldDiff                  9879 non-null   int64  
 11  blueExperienceDiff            9879 non-null   int64  
 12  blueCSPerMin                  9879 non-null   float64
 13  blu

In [12]:
df["blueWins"].value_counts()

0    4949
1    4930
Name: blueWins, dtype: int64

In [20]:
y = df["blueWins"]
X = df.drop("blueWins", axis=1)

ss = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=57)

X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

boost_model = XGBClassifier(random_state=57, objective="reg:logistic")

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [2, 3, 4, 5],
    'min_child_weight': [1, 2, 3, 4, 5, 6],
    'subsample': [0.4, 0.5, 0.6, 0.7],
    'n_estimators': [30, 50, 100]
}

gridsearch = GridSearchCV(boost_model, param_grid, cv=3, scoring="accuracy", n_jobs=1)
gridsearch.fit(X_train_ss, y_train)

best_parameters = gridsearch.best_params_

print("Best Parameters: ")
print(best_parameters)

Best Parameters: 
{'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.6}


In [21]:
train_pred = gridsearch.predict(X_train_ss)
test_pred = gridsearch.predict(X_test_ss)

train_acc = accuracy_score(y_train, train_pred)
train_f1 = f1_score(y_train, train_pred)

print("Training Scores")
print("Accuracy: ", train_acc)
print("F1: ", train_f1)

test_acc = accuracy_score(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred)

print("Test Scores")
print("Accuracy: ", test_acc)
print("F1: ", test_f1)

Training Scores
Accuracy:  0.7376164124713187
F1:  0.7310459324847814
Test Scores
Accuracy:  0.7279352226720648
F1:  0.7245901639344263


In [22]:
logreg = LogisticRegression(max_iter=5000).fit(X_train, y_train)

logreg_train = logreg.predict(X_train)
logreg_test = logreg.predict(X_test)

acc = accuracy_score(y_train, logreg_train)
f1 = f1_score(y_train, logreg_train)

print("Training Scores")
print("Accuracy: ", acc)
print("F1: ", f1)

acc = accuracy_score(y_test, logreg_test)
f1 = f1_score(y_test, logreg_test)

print("Testing Scores")
print("Accuracy: ", acc)
print("F1: ", f1)

Training Scores
Accuracy:  0.7358617897152112
F1:  0.7338501291989663
Testing Scores
Accuracy:  0.7311740890688259
F1:  0.7331189710610932
